## Làm sạch thư mục dowload của kaggle

In [ ]:
import os

path = "/kaggle/working/"
for file_name in os.listdir(path):
    file = path + file_name
    if os.path.isfile(file):
        print('Deleting file:', file)
        os.remove(file)

## Tải thư viện

In [ ]:
!pip install --no-dependencies --quiet streamlit
!pip install matplotlib
!pip install pandas
!pip install datetime
!pip install --upgrade pip setuptools
!pip install torch
!pip install transformers

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

In [ ]:
!pip install --quiet pyngrok

In [ ]:
!pip install --no-dependencies --quiet protobuf==3.20.*   #==4.21.12

In [ ]:
!pip install --no-dependencies --quiet validators

In [ ]:
!ngrok config add-authtoken 2hmcMJdnkF1N4BgmhfE4DMTi2el_2c7Khyg63zX7okMa2T5r5

## Chạy sản phẩm

In [ ]:
%%writefile config.json
{
    "test_1.csv": {
        "type": "line",
        "title": "Số lượng người dùng Facebook hoạt động hàng tháng trên toàn thế giới tính đến quý 3 năm 2020 (tính bằng triệu)",
        "column": "Số lượng người dùng tính bằng triệu"
    },
    "test_2.csv": {
        "type": "column",
        "title": "Hoa Kỳ: giá trị tài sản ròng ước tính của 20 người giàu nhất tính đến tháng 3 năm 2020 (tính bằng tỷ đô la Mỹ)",
        "column": "Giá trị ròng tính bằng tỷ đô la Mỹ"
    },
    "test_3.csv": {
        "type": "line",
        "title": "Doanh thu và thu nhập ròng của Facebook từ năm 2007 đến 2019 (triệu đô la Mỹ)",
        "column": "Số tiền tính bằng triệu USD"
    }
}


In [ ]:
%%writefile process.py
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset
import json
from random import randint
import pandas as pd
from transformers import GenerationConfig

path = "/kaggle/input/data-chart/data/"
def cleanAxisValue(value):
    if value == '-' or value == 'nan':
        return '0'
    cleanValue = value.replace('|', '').replace(',', '').replace('%', '').replace('*', '')
    return cleanValue

def preprocess_dataset(file_name, config):
    df = pd.read_csv(path + file_name)
    column = df.columns.tolist()
    column_input = ' | '.join(cleanAxisValue(column[i]) for i in range(len(column)))
    row_input = ' ; '.join(' | '.join(cleanAxisValue(str(df.loc[index_row, col])) for col in column) for index_row in range(len(df)))
    input = column_input + ' ; ' + row_input
    instruction = config[file_name]['title']
    result = {'instruction': instruction, 'input': input}
    return result

def load_model():
    #Load model
    base_model = "XMin08/Model_Llama2_new_v4"

    tokenizer = LlamaTokenizer.from_pretrained(base_model)
    model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )
#     model = LlamaForCausalLM.from_pretrained(
#         base_model,
#         torch_dtype=torch.float32,  # Sử dụng float32 cho CPU
#     ).to('cpu')  # Chuyển mô hình sang CPU
    
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2
    return model, tokenizer

def generate_output(data, model, tokenizer):
    prompt = """
                  Below is an instruction that describes a task, paired with an input that provides further context.
                  Write a response that appropriately completes the request.

    ### Instruction:\n{}\n\n### Input:\n{}\n\n### Response:
    """

    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        do_sample = True
    )

    prompt_sentence = prompt.format(data['instruction'], data['input'])
    inputs = tokenizer(prompt_sentence, return_tensors="pt")
    input_ids = inputs["input_ids"].to("cuda")
    generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=512,
            )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    predict = output.split("### Response:")[1].strip().split("</s>")[0].strip()
    return predict


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from process import preprocess_dataset, load_model, generate_output

# Đọc dữ liệu từ file CSV
@st.cache_resource  # Cache dữ liệu để tăng tốc độ xử lý
def load_data(file_path):
    return pd.read_csv(path + file_path)

# Hàm để đọc tệp cấu hình JSON
def read_config(file):
    with open(file, 'r', encoding='utf-8') as f:
        return json.load(f)

@st.cache_resource
def load_model_once():
    return load_model()

path = "/kaggle/input/data-chart/data/"
# Lấy danh sách các tệp CSV trong thư mục
csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]

# Load Model
model, tokenizer = load_model_once()

# Kiểm tra nếu không có tệp CSV nào
if not csv_files:
    st.error("Không có tệp CSV nào trong thư mục.")
else:
    # Tạo sidebar cho việc lựa chọn tệp CSV và các tùy chọn khác
    st.sidebar.title("Tùy chọn")
    selected_file = st.sidebar.selectbox('Chọn tệp CSV:', [file.split('.')[0] for file in csv_files])
    selected_file += '.csv'

    # Đọc dữ liệu từ tệp CSV được chọn
    data = load_data(selected_file)

    # Đọc cấu hình từ file config.json
    config = read_config('config.json')

    # Lấy thông tin cấu hình cho tệp CSV được chọn
    file_config = config.get(selected_file, {})
    chart_type = file_config.get('type', 'column')
    y_column = file_config.get('column', None)
    title = file_config.get('title', f'Tiêu đề của Biểu đồ {selected_file}')

    # Lấy tên cột đầu tiên làm trục x
    x_column = data.columns[0]

    # Tạo danh sách các cột (trừ cột đầu tiên) làm trục y
    y_columns = data.columns[1:]

    # Tiêu đề chính cho ứng dụng
    st.title('Hiển thị biểu đồ với Streamlit')

    # Hiển thị tiêu đề cho biểu đồ
    st.header(title)

    # Hiển thị DataFrame
    with st.expander("Hiển thị dữ liệu mẫu"):
        st.write(data)

    # Vẽ biểu đồ dựa trên lựa chọn
    st.subheader("Biểu đồ:")
    fig, ax = plt.subplots(figsize=(14, 7))  # Thay đổi kích thước biểu đồ

    if chart_type == 'line':
        # Vẽ biểu đồ đường cho mỗi cột trên trục y
        for column in y_columns:
            ax.plot(data[x_column], data[column], marker='o', label=column)
        ax.set_title(title)
    elif chart_type == 'column':
        # Vẽ biểu đồ cột cho mỗi cột trên trục y
        for column in y_columns:
            ax.bar(data[x_column], data[column], label=column)
        ax.set_title(title)

    ax.set_xlabel(x_column)
    ax.set_ylabel(y_column)
    plt.xticks(rotation=45, ha='right') 
    ax.legend()
    st.pyplot(fig)

    # Tạo một nút để hiển thị đoạn văn bản
    if st.button('Generate'):
        preprocess_data = preprocess_dataset(selected_file, config)
        predict = generate_output(preprocess_data, model, tokenizer)
        st.markdown(f"""
            ### Thông tin
            Output: {predict}
        """)


In [ ]:
# !streamlit run --server.port 80 my_app.py > /dev/null

import subprocess
subprocess.Popen(["streamlit", "run", "--server.port", "80", "app.py"])

In [ ]:
!ngrok http 80